In [1]:
from functools import partial

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.applications import resnet50
from keras.src.applications import ResNet50
from tqdm import tqdm_notebook as tqdm

from mobilenetv2.utils import split_functional_model

In [2]:
def normalize_img(img, lbl):
    """Normalizes images: `uint8` -> `float32`."""
    img = tf.image.resize_with_pad(img, 224, 224)
    img = resnet50.preprocess_input(img)
    return img, lbl

In [3]:
# load imagenet2012 dataset
validation_ds, metadata = tfds.load(
    'imagenet2012',
    split='validation',
    with_info=True,
    as_supervised=True,
)
validation_ds = validation_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
get_label_name = metadata.features['label'].int2str

In [4]:
def representative_dataset(head_network=None):
    # should be 100 to 500 according to documentation
    number_of_samples = 100
    if head_network is None:
        for data in validation_ds.batch(1).take(number_of_samples):
            yield [data[0]]
    else:
        interpreter = tf.lite.Interpreter(model_content=head_network)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]
        output_details = interpreter.get_output_details()[0]
        input_scale, input_zero_point = input_details["quantization"]
        output_scale, output_zero_point = output_details["quantization"]
        for data in validation_ds.batch(1).take(number_of_samples):
            test_image = data[0][0] / input_scale + input_zero_point
            test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
            interpreter.set_tensor(input_details["index"], test_image)
            interpreter.invoke()
            yield [((interpreter.get_tensor(output_details['index']) - output_zero_point) * output_scale).astype(
                "float32")]

In [5]:
def quantize_and_save_model(model, name, head_network=None):
    # Convert the model.
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = partial(representative_dataset, head_network)
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8  # or tf.uint8
    converter.inference_output_type = tf.int8  # or tf.uint8
    tflite_model = converter.convert()

    # Save the model.
    with open("models/" + name + '.tflite', 'wb') as f:
        f.write(tflite_model)
    return tflite_model

In [4]:
model = ResNet50(weights='imagenet')

In [7]:
print("Save full model")
_ = quantize_and_save_model(model, "full")

Save full model
INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpg21bj4f6\assets


INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpg21bj4f6\assets
C:\Users\danie\tuWien\10_semester\tensorflow\venv\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [8]:
print("Save partial models")
# skip full model with first and last index
for i in tqdm(range(1, 40)):
    head, tail = split_functional_model(model, i)
    head_quantized = quantize_and_save_model(head, "head/" + str(i))
    quantize_and_save_model(tail, "tail/" + str(i), head_quantized)

Save partial models


C:\Users\danie\AppData\Local\Temp\ipykernel_27420\2377953975.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(1, 75)):


  0%|          | 0/74 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmp9c3qb1mx\assets


INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmp9c3qb1mx\assets
C:\Users\danie\tuWien\10_semester\tensorflow\venv\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


...

INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpbt6a1c_n\assets
C:\Users\danie\tuWien\10_semester\tensorflow\venv\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpaog5qx62\assets


INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpaog5qx62\assets
C:\Users\danie\tuWien\10_semester\tensorflow\venv\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpjgp1aveg\assets


INFO:tensorflow:Assets written to: C:\Users\danie\AppData\Local\Temp\tmpjgp1aveg\assets
C:\Users\danie\tuWien\10_semester\tensorflow\venv\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [4]:
# prediction on the full 32 bit floating point model
it = iter(validation_ds)
for _ in range(8):
    image, label = next(it)
preds = model.predict(tf.stack([image]))
print('32 bit fp prediction:', resnet50.decode_predictions(preds, top=5)[0])

NameError: name 'model' is not defined

In [10]:
# prediction on the TensorFlow Lite 8 bit quantized model
interpreter = tf.lite.Interpreter(model_path="models/full.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

# convert image to 8 bit
input_scale, input_zero_point = input_details["quantization"]
test_image = image / input_scale + input_zero_point
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
interpreter.set_tensor(input_details["index"], test_image)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details['index'])
print('8 bit quantized prediction:', resnet50.decode_predictions(output_data, top=5)[0])

8 bit quantized prediction: [('n02086240', 'Shih-Tzu', 4), ('n02098413', 'Lhasa', -113), ('n02098105', 'soft-coated_wheaten_terrier', -114), ('n02096437', 'Dandie_Dinmont', -123), ('n02088094', 'Afghan_hound', -123)]


In [11]:
def show_results(split_at):
    # prediction on the TensorFlow Lite 8 bit split quantized model
    head = tf.lite.Interpreter(model_path="models/head/" + str(split_at) + ".tflite")
    tail = tf.lite.Interpreter(model_path="models/tail/" + str(split_at) + ".tflite")
    head.allocate_tensors()
    tail.allocate_tensors()

    # Get input and output tensors from head network.
    input_details = head.get_input_details()[0]
    output_details = head.get_output_details()[0]

    #convert image
    input_scale, input_zero_point = input_details["quantization"]
    output_scale, output_zero_point = output_details["quantization"]
    test_image = image / input_scale + input_zero_point
    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])

    # invoke head network
    head.set_tensor(input_details["index"], test_image)
    head.invoke()
    intermediate = head.get_tensor(output_details['index'])

    # rescale tensor
    intermediate_float = ((intermediate - output_zero_point) * output_scale).astype("float32")

    # Get input and output tensors from tail network and convert tensor.
    input_details = tail.get_input_details()[0]
    output_details = tail.get_output_details()[0]
    input_scale, input_zero_point = input_details["quantization"]
    intermediate_int = (intermediate_float / input_scale + input_zero_point).astype(input_details["dtype"])

    # invoke tail network
    tail.set_tensor(input_details["index"], intermediate_int)
    tail.invoke()
    output_data = tail.get_tensor(output_details['index'])
    print('8 bit split at ' + str(split_at) + ' prediction:', mobilenet_v2.decode_predictions(output_data, top=5)[0])

In [12]:
for i in range(1, 40):
    show_results(i)

8 bit split at 1 prediction: [('n02086240', 'Shih-Tzu', -20), ('n02098105', 'soft-coated_wheaten_terrier', -105), ('n02098413', 'Lhasa', -110), ('n02096177', 'cairn', -121), ('n02090721', 'Irish_wolfhound', -124)]
8 bit split at 2 prediction: [('n02086240', 'Shih-Tzu', -91), ('n02098105', 'soft-coated_wheaten_terrier', -102), ('n02090721', 'Irish_wolfhound', -111), ('n02095314', 'wire-haired_fox_terrier', -116), ('n02091635', 'otterhound', -118)]
8 bit split at 3 prediction: [('n03598930', 'jigsaw_puzzle', 53), ('n03782006', 'monitor', -114), ('n03661043', 'library', -122), ('n04418357', 'theater_curtain', -125), ('n04548362', 'wallet', -126)]
8 bit split at 4 prediction: [('n02086240', 'Shih-Tzu', -36), ('n02098413', 'Lhasa', -107), ('n02098105', 'soft-coated_wheaten_terrier', -115), ('n02096177', 'cairn', -118), ('n02096437', 'Dandie_Dinmont', -119)]
8 bit split at 5 prediction: [('n02086240', 'Shih-Tzu', -30), ('n02098105', 'soft-coated_wheaten_terrier', -93), ('n02098413', 'Lhasa',

In [6]:
# prediction on the TensorFlow Lite 8 bit split quantized model
split_at = 30
head = tf.lite.Interpreter(model_path="models/head/" + str(split_at) + ".tflite")
tail = tf.lite.Interpreter(model_path="models/tail/" + str(split_at) + ".tflite")
head.allocate_tensors()
tail.allocate_tensors()

# Get input and output tensors from head network.
input_details = head.get_input_details()[0]
output_details = head.get_output_details()[0]

#convert image
input_scale, input_zero_point = input_details["quantization"]
output_scale, output_zero_point = output_details["quantization"]
test_image = image / input_scale + input_zero_point
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])

# invoke head network
head.set_tensor(input_details["index"], test_image)
head.invoke()
intermediate = head.get_tensor(output_details['index'])

# rescale tensor
intermediate_float = ((intermediate - output_zero_point) * output_scale).astype("float32")

# Get input and output tensors from tail network and convert tensor.
input_details = tail.get_input_details()[0]
output_details = tail.get_output_details()[0]
input_scale, input_zero_point = input_details["quantization"]
intermediate_int = (intermediate_float / input_scale + input_zero_point).astype(input_details["dtype"])

# invoke tail network
tail.set_tensor(input_details["index"], intermediate_int)
tail.invoke()
output_data = tail.get_tensor(output_details['index'])
print('8 bit split at ' + str(split_at) + ' prediction:', resnet50.decode_predictions(output_data, top=5)[0])

8 bit split at 30 prediction: [('n02086240', 'Shih-Tzu', -7), ('n02098413', 'Lhasa', -111), ('n02098105', 'soft-coated_wheaten_terrier', -114), ('n02096437', 'Dandie_Dinmont', -121), ('n02096177', 'cairn', -122)]


In [86]:
get_label_name(label)

'n02086240'

In [90]:
[label_id for label_id, _, _ in resnet50.decode_predictions(output_data, top=5)[0]]

['n02086240', 'n02098413', 'n02098105', 'n02096437', 'n02096177']

In [89]:
resnet50.decode_predictions(output_data, top=5)[0]

[('n02086240', 'Shih-Tzu', -7),
 ('n02098413', 'Lhasa', -111),
 ('n02098105', 'soft-coated_wheaten_terrier', -114),
 ('n02096437', 'Dandie_Dinmont', -121),
 ('n02096177', 'cairn', -122)]

In [72]:
serialized_tensor_bytes = tf.io.serialize_tensor(intermediate_float).numpy()

In [74]:
deserialized_tensor = tf.io.parse_tensor(serialized_tensor_bytes, out_type=tf.float32)

In [76]:
deserialized_tensor

<tf.Tensor: shape=(1, 28, 28, 192), dtype=float32, numpy=
array([[[[ 3.3855433,  0.2604264,  1.8229848, ...,  0.       ,
           3.1251168,  2.604264 ],
         [ 5.208528 , -1.5625584, -1.8229848, ..., -1.0417056,
          -1.302132 , -0.7812792],
         [ 5.7293806, -2.0834112, -2.8646903, ...,  0.5208528,
          -2.604264 , -1.5625584],
         ...,
         [ 5.4689546, -1.8229848, -3.3855433, ...,  0.7812792,
          -2.8646903, -1.8229848],
         [ 5.7293806, -1.8229848, -2.0834112, ..., -0.2604264,
          -3.1251168, -1.302132 ],
         [-1.0417056,  2.8646903, -2.604264 , ...,  3.3855433,
           2.604264 ,  3.3855433]],

        [[ 2.8646903, -1.0417056,  0.2604264, ..., -1.5625584,
          -2.0834112, -3.906396 ],
         [ 6.51066  , -1.8229848, -1.302132 , ..., -3.906396 ,
          -5.208528 , -2.8646903],
         [ 5.989807 , -2.0834112, -1.5625584, ..., -2.604264 ,
          -4.427249 , -3.3855433],
         ...,
         [ 6.7710867, -2.60426